In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text

LOG = rts.utils.get_logger()

In [8]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
# outpath = rts.io.media.trim_media(video_path, 300, 310, Path(OUTDIR) / 'test_trim.mp4')


In [5]:
import ffmpeg

In [86]:
from typing import Dict, List, Tuple, Any, Optional, Union

def trim(input_path: Union[str, Path], output_path: Union[str, Path], start: int = 30, end: int = 60) -> Optional[Any]:
    # https://github.com/kkroening/ffmpeg-python/issues/184
    input_stream = ffmpeg.input(str(input_path), hide_banner=None)
    vid = (
        input_stream.video
        .trim(start=start, end=end)
        .setpts('PTS-STARTPTS')
    )
    aud = (
        input_stream.audio
        .filter_('atrim', start=start, end=end)
        .filter_('asetpts', 'PTS-STARTPTS')
    )

    joined = ffmpeg.concat(vid, aud, v=1, a=1).node
    print(joined)
    output = ffmpeg.output(joined[0], joined[1], filename='pipe:', 
                          format='webm', vcodec='libvpx-vp9', acodec='libopus')
    print(output)
    output = output.overwrite_output()
    return output.run(capture_stdout=True)

In [87]:
res = trim(video_path, '-', 300, 310)
print(res)
rts.io.media.get_media_info(outpath)

concat(a=1, n=1, v=1) <15897ff85f15>
output(acodec='libopus', filename='pipe:', format='webm', vcodec='libvpx-vp9')[None] <16133d3f38a6>


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/media/data/rts/archive/0/0/0/ZB001000/ZB001000.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf59.27.100
  Duration: 00:28:27.05, start: 0.000000, bitrate: 525 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 768x576 [SAR 1:1 DAR 4:3], 441 kb/s, 25 fps, 25 tbr, 12800 tbn, 50 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, mono, fltp, 80 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:0 (h264) -> trim
  Stream #0:1 (aac) -> atrim
  concat:out:v0 -> Stream #0:0 (libvpx-vp9)
  concat:out:a0 -> Stream #0:1 (libopus)
Press [q] to stop, [?] for help
[libvpx-vp9 @ 0x555b7b4a42c0] v1.8.2
[libopus @ 0x555b7b473c80] No bit rate set. Defaulting to 64000 bps.
Output #0, webm, to 'pipe:':
  Metadata:
    major_bra

(b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x11M\x9bt\xacM\xbb\x8bS\xab\x84\x15I\xa9fS\xac\x81\xe5M\xbb\x8cS\xab\x84\x16T\xaekS\xac\x82\x01\x1cM\xbb\x8cS\xab\x84\x12T\xc3gS\xac\x82\x01\xc1\xec\x01\x00\x00\x00\x00\x00\x00\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

frame=  250 fps= 14 q=0.0 Lsize=     337kB time=00:00:10.01 bitrate= 276.0kbits/s speed=0.579x    
video:249kB audio:82kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.818857%


{'path': 'data/test_trim.mp4',
 'filesize': 638924,
 'human_filesize': '623.95 KiB',
 'duration': 10.022,
 'video': {'codec': 'h264',
  'width': 768,
  'height': 576,
  'framerate': '25',
  'bitrate': 434237,
  'human_bitrate': '434.24K'},
 'audio': {'codec': 'aac',
  'channels': 1,
  'sample_rate': 48000,
  'bitrate': 69543,
  'human_bitrate': '69.54K'}}

In [ ]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

# df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
# sample_df = rts.metadata.get_one_percent_sample(df)
# # rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# # rts.metadata.export_metadata_stats(df, OUTDIR)
# sample_df.head()

# medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)
# clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

clip_df = rts.metadata.build_clips_df(LOCAL_VIDEOS)
# rts.utils.dataframe_to_hdf5(METADATA, 'clips', clip_df)

In [ ]:
rts.metadata.build_clip_index(clip_df)

In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [ ]:
medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)

In [ ]:
clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

In [ ]:
df = rts.metadata.build_clips_df(LOCAL_VIDEOS)

In [ ]:
atlases = rts.metadata.create_clip_texture_atlases(df, LOCAL_RTS_DATA,
                                                   '0', # folder name
                                                   tile_size=64,
                                                   flip=True,
                                                   no_border=True, 
                                                   format='jpg')

In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [ ]:
test_im_path = os.path.join(OUTDIR, 'test_image.jpg')
im = Image.open(test_im_path)
# rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# audio_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000_audio.aac')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = rts.io.media.trim_media(video_path, 30, 40, Path(OUTDIR) / 'test_trim.mp4')
rts.io.media.get_media_info(outpath)

In [ ]:
30*48000

In [ ]:
transcript = rts.pipeline.transcribe_media(
    OUTDIR,
    None, 
    force=False
)

t = rts.features.text.find_locations(transcript)

In [ ]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [ ]:
imgs = list(rts.utils.recursive_glob(Path(OUTDIR) / 'images' / 'original', '.jpg'))
imgs

In [ ]:
rts.io.media.create_square_atlases(
    imgs,
    Path(OUTDIR) / 'atlas',
    width=512,
    max_tile_size=128,
    no_border=True,
    flip=True,
    atlas_prefix='small_atlas',
    bg_color=(255, 255, 255, 255)
)

In [ ]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=False, force_trans=False)

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080") # or another location where your Weaviate instance is running

client.schema.get() # get the full schema as example

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
swiss_cities['Genève']

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})